# Kleines Skript zum Überprüfen von Links in CSV-Dateien

by Claudia Wordsworth

Mit diesem Skript können Links in CSV-Dateien auf ihren Responsestatus überprüft werden, um nicht mehr funktionierende Links, z. Bsp. in Readinglists zu finden. Alle Fehlermeldungen werden abschließend in einem CSV-Report geschrieben und per E-Mail an eine gewünschte E-Mail-Adresse geschickt. Eine kleine Info-E-Mail wird verschickt, wenn alle Links funktionieren. 

## 1. Importieren aller notwendigen Bibliotheken

In [5]:
import pandas as pd #Einlesen und erstellen der CSV-Datei
import requests #Statusabfrage der Links
from email.message import EmailMessage #Zum Erstellen der E-Mails
import smtplib # Zum Versenden der E-Mails
from datetime import datetime #Zum Generieren des Zeitstempels für Report

## 2. Erstellen aller notwendigen Funktionen

Für das Versenden des Ergebnisses der Überprüfung per E-Mail werden zwei Funktionen geschrieben. Die erste, die bei Fehlermeldungen den Report als Anhang verschickt und eine zweite für eine einfache Info-E-Mail, falls alles ok ist.

In [8]:
#E-Mail-Funktion mit Report als Anhang:
#Als erstes erstelle ich eine Funktion, die eine E-Mail mit CSV-Anhang an die Ziel-E-Mail-Addresse sendet:

def send_email_alert(sender, recipient, report):
    
    from email.message import EmailMessage
    import smtplib
    #Das richtige MIME-Format wird anhand der Dateiendung erstellt
    import mimetypes
    
#Kopf der E-Mail mit den Variablen für den Sender und Empfänger. Diese werden beim Funktionsaufruf angegeben.

    msg = EmailMessage()
    msg["Subject"] = "Broken Links in Reading List"
    msg['From'] = sender
    msg['To'] = recipient

#Nachricht und Anhang. Der Report wird als Info beim Funktionsaufruf angegeben.

    date = datetime.now().strftime('%Y-%m-%d') 
    
    #Text in der E-Mail
    msg.set_content("Dear Team, \nsome of the links are not working. Please have a look!")
    
    #Damit wird die Dateiendung überprüft, so dass die Datei in der E-Mail im richtigen Format übermittelt wird.
    
    ctype, encoding = mimetypes.guess_type(report)
    maintype, subtype = ctype.split('/', 1)
    
    #Datei wird an die E-Mail angehängt
    with open(report, "rb") as attachment:
        msg.add_attachment(
            attachment.read(), maintype=maintype, subtype=subtype, filename= date + "_broken_links_report")
    
#Verbindung zum E-Mail-Server wird aufgebaut und E-Mail abgeschickt

    with smtplib.SMTP('smtp.web.de', 587) as smtplib:
        smtplib.ehlo()
        smtplib.starttls()
        smtplib.login(sender, 'password')
        smtplib.send_message(msg)

In [2]:
#Funktion für eine Gute-Nachricht-E-Mail, im Falle, dass alle Links in Ordnung sind: 

def all_ok(sender, recipient):

    from email.message import EmailMessage
    import smtplib

#Kopf der E-Mail mit den Variablen für den Sender und Empfänger. Diese werden beim Funktionsaufruf angegeben.

    msg = EmailMessage()
    msg['Subject'] = 'All links are ok - EOM'
    msg['From'] = sender
    msg['To'] = recipient

#Nachricht bleibt leer.

    msg.set_content(" ")


# Verbindung zum E-Mail-Server wird aufgebaut und E-Mail abgeschickt.

    with smtplib.SMTP('smtp.web.de', 587) as smtplib:
        smtplib.ehlo()
        smtplib.starttls()
        smtplib.login(sender, 'password')
        smtplib.send_message(msg)

Gibt es Fehlermeldungen, werden diese mit der dazugehörigen URL in einer Liste gesammelt. Diese wird mit dieser Funktion in eine Tabelle umgewandelt und als CSV-Datei gespeichert. Zum Schluss wird die E-Mail-Funktion aufgerufen.

In [3]:
#Datei wird als Tabelle geschrieben und als CSV-Datei gespeichert:

def Create_report(report):
    
    date = datetime.now().strftime('%Y-%m-%d') #Das Datum soll im Reportnamen generiert werden
    
    df = pd.DataFrame(report)           
    reportdf = df[0].str.split(';', expand=True)            
    reportdf.columns=['Item Links', 'Error Status']
    reportdf.to_csv(date + '_broken_links_report.csv', index = False)
    
    send_email_alert('email@web.de', 'email@web.de', date + '_broken_links_report.csv')

# 3. Beginn des Link-Überprüfungsprogramms    

Mit Pandas wird die gewünschte Datei eingelesen. Die Spalte mit den enthaltenen Links wird isoliert und alle Zeilen entfernt, in denen sich keine URL befindet.


In [6]:
#Einlesen der Datei

reading_list = pd.read_csv('Beispiel_Readinglist.csv')

links = reading_list['Item Link'].dropna()

Hier beginnt die eigentliche Überprüfung. Da ich nur den Status überprüfen und nicht die Seite öffnen möchte, benutze ich request.head() anstelle von request.get(). Bei größeren Dateien könnte das Zeit sparen. 
Es werden nach HTTP-Ausnahmen (Fehlercode: 400 - 500) und weitere Ausnahmen gesucht. Ich habe aber bei den HTTP-Ausnahmen noch die Möglichkeit gelassen, nur nach den Statuscodes zu fragen, falls es gewollt ist und die eigentliche Fehlermeldung nicht benötigt wird.
Alle Fehlermeldungen werden in einer Liste gespeichert. Ist die Liste leer, wird die E-Mail-Funktion für die Gute-Nachricht-E-Mail aufgerufen. Ansonsten wird die CSV-Datei erstellt und verschickt. 

In [7]:
#Mit dem Moduel "request" wird Statusabfrage gestartet und Ergebnisse in einer neuen Liste gespeichert:

broken_links = []

for url in links:
    
    try:
        response = requests.head(url)
        response.raise_for_status()
        
    # Alle Ausnahmen werden in einer Liste gespeichert:
    except requests.exceptions.HTTPError as http_error:
        
           #error_status = response.status_code
            
           broken_links.append(f'{url}; {http_error}')
            
    except requests.exceptions.RequestException as other_error:
        
           broken_links.append(f'{url}; {other_error}')
            
#Falls es keine Fehlermeldungen gibt, wird die Gute-Nachricht-E-Mail verschickt: 
            
if len(broken_links) == 0:
    all_ok('email@web.de', 'email@web.de')
                                
#Gibt es Fehlermeldungen, wird die Liste in eine CSV-Datei umgewandelt und per E-Mail verschickt:

Create_report(broken_links)



## Fertig!

Am Ende sollte man eine E-Mail erhalten haben.